# Finetuning GPT-3.5 - CW Coherence.ipynb

Sources: https://blog.futuresmart.ai/fine-tuning-gpt-35-a-step-by-step-guide

In [1]:
# Whether this notebook is random 50% split 1 or random 50% split 2
this_nb = 'random_50_LP_2'


In [2]:
import pandas as pd
import openai


In [3]:

full_human_data = pd.read_csv("full_human_data.csv")
full_human_data.head(5)


,model_task_method,conversation_number,response,coherence_1_incoherent_10_very_coherent
0,td3_cw_direct_prompting_responses,1,Learning to do a handstand is a fun activity f...,1
1,td3_cw_direct_prompting_responses,2,The hawk was used to hunting what he needed fo...,7
2,td3_cw_direct_prompting_responses,3,I love the smell of roasting almonds in the ki...,1
3,td3_cw_direct_prompting_responses,4,Ralph's bedroom was routinely filled up with s...,10
4,td3_cw_direct_prompting_responses,5,Joe had a unique way of dealing with the hospi...,4


In [4]:
# Set seed to 201
import numpy as np
np.random.seed(201)

# Draw 50 numbers from 1 to 100
random_50 = np.random.choice(100, 50, replace=False)

print(random_50)


[91 57 19 66 30 74 86 67 29 83  7 31 13 89 65  1 62 93 44 76 92 85 59  6
 11 45 22  4 25 34 12 39 49 47 69 18 51 97 64 55 46 10 58 82 79  3 96 84
 70 14]


In [5]:
# Split
if this_nb == 'random_50_LP_1':
    this_notebook_train_validate = full_human_data.query('conversation_number in @random_50')
    this_notebook_predictions = full_human_data.query('conversation_number not in @random_50')
else:
    this_notebook_train_validate = full_human_data.query('conversation_number not in @random_50')
    this_notebook_predictions = full_human_data.query('conversation_number in @random_50')


In [6]:
# Save this notebook's predictions
this_notebook_predictions.to_csv('items_to_predict_using_' + this_nb + '.csv', index=False)


In [7]:
# Print responses for 5 observations
for i in range(5):
    print(this_notebook_train_validate.iloc[i, 2])
    print('')


The hawk was used to hunting what he needed for food, but one day he decided to try and be friends with the ground squirrels in the park. He thought it would be a good way to pass the time and to fill his empty belly. He went out of his way to make nice with the squirrels, but they ran away from him. The hawk didn't understand why the ground squirrels didn't want to be his friend.
The hawk was confused, but he soon learned an important lesson. Just like the ground squirrels, if I don't like something, I'll stay away from it. I may not understand what other people find attractive or interesting, but it's better to keep my distance and keep my opinion to myself. The lesson the hawk learned was to respect the boundaries of those around him.

Joe had a unique way of dealing with the hospital noise. Instead of relying on traditional sound volume reducers, like earplugs and noise-cancelling headphones, Joe opted for a more creative way to reduce the sound. He decided to construct a makeshift

### Format Dataset

In [8]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"Score": "' + str(row['coherence_1_incoherent_10_very_coherent']) + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['response'] + "\nYour Task: Rate the coherence of the passage above on a scale of 1 to 10, 1 being incoherent and 10 being very coherent. If the passage is two unconnected pieces give it a score of 1. If it has sentences that seem to be randomly inserted, has abrupt change in characters, or has abrupt change in setting, give it a low score. If it has a continuous setting or characters and seems plausible, give it a high score. If it is as coherent as you believe is possible, give it a score of 10."},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data


In [9]:
# Convert entire dataset
dataset = this_notebook_train_validate
converted_data = convert_to_gpt35_format(dataset)
converted_data[0]['messages']


[{'role': 'user',
  'content': "The hawk was used to hunting what he needed for food, but one day he decided to try and be friends with the ground squirrels in the park. He thought it would be a good way to pass the time and to fill his empty belly. He went out of his way to make nice with the squirrels, but they ran away from him. The hawk didn't understand why the ground squirrels didn't want to be his friend.\nThe hawk was confused, but he soon learned an important lesson. Just like the ground squirrels, if I don't like something, I'll stay away from it. I may not understand what other people find attractive or interesting, but it's better to keep my distance and keep my opinion to myself. The lesson the hawk learned was to respect the boundaries of those around him.\nYour Task: Rate the coherence of the passage above on a scale of 1 to 10, 1 being incoherent and 10 being very coherent. If the passage is two unconnected pieces give it a score of 1. If it has sentences that seem to b

In [10]:
import json
json.loads(converted_data[0]['messages'][-1]['content'])


{'Score': '7'}

Create Train and Val Set

In [11]:
# Print unique values of model_task_method
this_notebook_train_validate['model_task_method'].unique()


array(['td3_cw_direct_prompting_responses',
       'td3_cw_zero_shot_cot_responses',
       'td3_cw_ape_zero_shot_cot_responses',
       'td3_cw_least_to_most_responses',
       'td3_cw_manual_few_shot_responses', 'td3_cw_manual_cot_responses',
       'td3_cw_tree_of_thought_responses', 'td3_cw_self_refine_responses',
       'gpt4_cw_direct_prompting_responses',
       'gpt4_cw_zero_shot_cot_responses',
       'gpt4_cw_ape_zero_shot_cot_responses',
       'gpt4_cw_tree_of_thought_responses',
       'gpt4_cw_self_refine_responses', 'gpt4_cw_least_to_most_responses',
       'gpt4_cw_manual_few_shot_responses',
       'gpt4_cw_manual_cot_responses'], dtype=object)

In [12]:
from sklearn.model_selection import train_test_split

# Stratified splitting on model_task_method.
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=this_notebook_train_validate['model_task_method'],
    random_state=42  # for reproducibility
)


In [13]:
type(train_data[0])


dict

In [14]:
print(len(train_data))
print(len(val_data))


640
160


Create JSONL file

In [15]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train_" + this_nb + ".jsonl"
validation_file_name = "val_" + this_nb + ".jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)


In [16]:
# # Get and set API key
# with open('C:/Users/ijyli/Documents/OpenAI/anlp23-project.txt', 'r') as file:
#     api_key = file.read()
# openai.api_key = api_key


In [17]:

# Get and set API key
with open('C:/Users/ijyli/Documents/OpenAI/anlp23-project.txt', 'r') as file:
    my_api_key = file.read()

#from openai import OpenAI
from openai import OpenAI
client = OpenAI(api_key=my_api_key)


Upload Training and Validation File

In [18]:
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)


Training file id: file-Q7sVj7gAhOajH3a8ImA2WtK4
Validation file id: file-NN6qP50pZLQUnHVD8tA7KWqq


Create Finetuning Job

In [19]:
suffix_name = this_nb

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response


FineTuningJob(id='ftjob-nqlGEuKrF6lcCsfoPEDlJDKo', created_at=1701290431, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-MdSScMkbTK60fqqIP4cio0Kx', result_files=[], status='validating_files', trained_tokens=None, training_file='file-Q7sVj7gAhOajH3a8ImA2WtK4', validation_file='file-NN6qP50pZLQUnHVD8tA7KWqq')